In [ ]:
# %%
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Load Data from the Database ---
DB_NAME = "stock_data.db"
STOCK_TICKER = "AAPL" # Make sure this matches the table name

# Connect to the database
conn = sqlite3.connect(DB_NAME)

# Load the data using a SQL query into a pandas DataFrame
# "SELECT * FROM AAPL" means "get all columns from the AAPL table"
query = f"SELECT * FROM {STOCK_TICKER}"
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Teaching Moment: The 'Date' column was stored as text. We need to convert it
# back to a proper datetime format to work with it for plotting and analysis.
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True) # Set the Date as the index of the DataFrame

print("Data loaded successfully. Here's the information about our dataset:")
df.info()

# Display the first few rows
df.head()

In [ ]:
# %%
# .describe() gives you a summary of the main statistics for each numerical column
# (count, mean, standard deviation, min, max, etc.)
print("Statistical Summary:")
df.describe()

In [ ]:
# %%
# Check for any missing values in the dataset
print("Missing values in each column:")
print(df.isnull().sum())

In [ ]:
# %%
# Set the style for our plots
plt.style.use('seaborn-v0_8-whitegrid')

# Create the plot
plt.figure(figsize=(14, 7))
plt.plot(df['Close'], label=f'{STOCK_TICKER} Close Price')

# Add titles and labels for clarity
plt.title(f'{STOCK_TICKER} Historical Close Price', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend()
plt.show()

In [ ]:
# %%
# Calculate the 50-day and 200-day Simple Moving Averages (SMA)
df['SMA_50'] = df['Close'].rolling(window=50).mean()
df['SMA_200'] = df['Close'].rolling(window=200).mean()

# Plot the Close price along with the moving averages
plt.figure(figsize=(14, 7))
plt.plot(df['Close'], label='Close Price')
plt.plot(df['SMA_50'], label='50-Day Moving Average', color='orange')
plt.plot(df['SMA_200'], label='200-Day Moving Average', color='red')

plt.title(f'{STOCK_TICKER} Close Price with Moving Averages', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend()
plt.show()

In [ ]:
# %%
# --- 3. Prepare Data for Modeling ---

# We want to predict the next day's 'Close' price.
# So, our 'target' or 'y' is the 'Close' price shifted by one day.
df['Prediction'] = df['Close'].shift(-1)

# Our 'features' or 'X' are the columns we'll use for prediction.
# Let's drop the original columns that aren't numeric features.
# And we also drop the 'Prediction' column from our feature set.
X = df.drop(['Prediction', 'Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)

# The target variable is the 'Prediction' column
y = df['Prediction']

# The shift() and rolling() operations create NaN (Not a Number) values.
# We must remove them before training the model.
print(f"Original shape: {X.shape}")
original_X_index = X.index # Keep track of dates before dropping NaNs

# Drop rows with NaN values from both X and y
X = X.dropna()
y = y.loc[X.index] # Make sure y has the same rows as X

print(f"Shape after dropping NaNs: {X.shape}")

X.head()

In [ ]:
# %%
from sklearn.model_selection import train_test_split

# We will not split randomly, but by date. Let's take the last 20% of the data for testing.
test_size = 0.2
split_index = int(len(X) * (1 - test_size))

# Split the data chronologically
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

In [ ]:
# %%
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Create an instance of the Linear Regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

print("Model training complete.")

In [ ]:
# %%
# Make predictions on the test data
predictions = model.predict(X_test)

# Calculate the error. We'll use Root Mean Squared Error (RMSE).
# This tells us, on average, how many dollars off our prediction was.
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")

# For context, let's see the average stock price in the test set
print(f"Average price in test set: ${y_test.mean():.2f}")

In [ ]:
# %%
# Create a new DataFrame for plotting
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions}, index=y_test.index)

# Plot the actual vs. predicted prices
plt.figure(figsize=(14, 7))
plt.plot(results['Actual'], label='Actual Price')
plt.plot(results['Predicted'], label='Predicted Price', color='red', linestyle='--')
plt.title('Actual vs. Predicted Stock Prices', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend()
plt.show()